In [44]:
import pandas as pd
import numpy as np
import re
import string
import os
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web
init_notebook_mode(connected=True)

path = '/Users/deena/Documents/Spring_Module2/Data_viz/Project/'

In [45]:
df = pd.read_csv(path + 'zomato.csv',encoding = "ISO-8859-1")

In [46]:
df.head(1)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314


In [47]:
country_code_mapping = {
1: 'India',
14: 'Australia',
30: 'Brazil',
37: 'Canada',
94: 'Indonesia',
148: 'New Zealand',
162: 'Phillipines',
166: 'Qatar',
184: 'Singapore',
189: 'South Africa',
191: 'Sri Lanka',
208: 'Turkey',
214: 'UAE',
215: 'United Kingdom',
216: 'United States'
}

df['City_Country'] = df["City"] + ', ' + df["Country Code"].map(country_code_mapping)

In [48]:
df_city = df.groupby('City_Country').mean().reset_index()[['City_Country', 'Price range', 'Aggregate rating']]
df_city.head()

,City_Country,Price range,Aggregate rating
0,"Abu Dhabi, UAE",3.300000,4.300000
1,"Agra, India",2.650000,3.965000
2,"Ahmedabad, India",2.571429,4.161905
3,"Albany, United States",1.700000,3.555000
4,"Allahabad, India",2.650000,3.395000


In [49]:
data = []

for city in df_city['City_Country']:
    data.append(go.Bar(x=['Average Price Range', 'Average Rating'],
                        y=df_city[df_city.City_Country == city][['Price range', 'Aggregate rating']].values.tolist()[0],
                        name= city,
                       visible=False,
                       text = [round(n,2) for n in df_city[df_city.City_Country == city][['Price range', 'Aggregate rating']].values.tolist()[0]],
                       textposition='auto'
                        ))

buttons = []

for row, city in df_city.City_Country.iteritems():
    condition = [False]*len(df_city)
    condition[row] = True
    buttons.append(dict(label = city,
                 method = 'update',
                 args = [{'visible': condition},
                         {'title': 'Restaurant Summaries for City: ' + city,
                          }]))

updatemenus=list([
    dict(
        buttons = buttons,
        pad = {'r': 0, 't': 10},
        #x = 0.1,
        #xanchor = 'left',
       # y = 1.0,
        yanchor = 'top',
        bgcolor = 'AAAAAA',
        active = 99,
        bordercolor = '#FFFFFF',
        font = dict(size=11, color='#000000'))])

layout = dict(title='Restaurant Summaries for Cities', showlegend=False,
              updatemenus=updatemenus)

figure = dict(data=data, layout=layout)
#plotly.offline.plot(figure)
iplot(figure)

In [50]:
temp = pd.concat([df, df['Cuisines'].str.split(', ', expand=True)], axis=1)

df_cuisines = pd.melt(temp, id_vars=df.columns, 
                  var_name="Col", value_name="Cuisine_split")

df_cuisines = df_cuisines.groupby(['City_Country', 'Cuisine_split']).count().reset_index()[['City_Country', 'Cuisine_split', 'Col']]

In [52]:
data = []

for city in df_cuisines['City_Country'].drop_duplicates():
    data.append(go.Bar(x=df_cuisines[df_cuisines.City_Country == city]['Cuisine_split'].values.tolist(),
                        y=df_cuisines[df_cuisines.City_Country == city]['Col'].values.tolist(),
                        name= city,
                       visible=False,
                       text = df_cuisines[df_cuisines.City_Country == city]['Col'].values.tolist(),
                       textposition='auto'
                        ))

buttons = []

for row, city in df_cuisines.City_Country.drop_duplicates().reset_index()['City_Country'].iteritems():
    condition = [False]*len(df_cuisines.City_Country.drop_duplicates())
    condition[row] = True
    buttons.append(dict(label = city,
                 method = 'update',
                 args = [{'visible': condition},
                         {'title': 'Cuisines Summaries for City: ' + city,
                          }]))

updatemenus=list([
    dict(
        buttons = buttons,
        pad = {'r': 0, 't': 10},
        #x = 0.1,
        #xanchor = 'left',
       # y = 1.0,
        yanchor = 'top',
        bgcolor = 'AAAAAA',
        active = 99,
        bordercolor = '#FFFFFF',
        font = dict(size=11, color='#000000'))])

layout = dict(title='Cuisines Summaries for Cities', showlegend=False,
              updatemenus=updatemenus)

figure = dict(data=data, layout=layout)
#plotly.offline.plot(figure)
iplot(figure)